# Isochrone of Milan's Infrastructures
_"An isochrone map in urban planning is a map that depicts the area accessible from a point within a certain time/distance threshold."_

(Original inspiration: https://betterprogramming.pub/creating-a-traveling-distance-map-for-a-whole-city-with-python-f2e063272832)

## Setup

### Packages & Settings

In [ ]:
# to store and process data
import pandas as pd
import geopandas as gpd
import numpy as np
from tqdm import tqdm

# to create city's street graph and work with it
from geopy.point import Point
from geopy.distance import geodesic
import osmnx as ox
import networkx as nx

# to plot isochrones
import folium
import branca.colormap

# to handle data import/export
from urllib.request import urlopen
from io import BytesIO
import pickle

In [ ]:
tqdm.pandas()

### Utils

In [ ]:
class DictSmallest(dict):
    '''
    A dictionary that only updates a value if the provided one is smaller than the one already present (or if the value is currently not present)
    '''

    def __setitem__(self, key, value):
        if (key not in self) or (key in self and self[key] > value):
            dict.__setitem__(self, key, value)

    def update(self, dict):
        for key, value in dict.items():
            if (key not in self) or (key in self and self[key] > value):
                self[key] =  value

In [ ]:
def perc(num, den):
    return round(num/den*100, 2)

## Data Import & Preparation

### Milan Open Data

In [ ]:
def import_geojson(url):

    file = BytesIO(urlopen(url).read())
    gdf = gpd.read_file(file)

    return gdf

In [ ]:
def keys_relationship(df, colA:str, colB:str):
    '''
    Function to quickly analyze the relationship between two keys within the same dataset
    '''

    count1 = df.groupby(colA).nunique()[colB].sort_values(ascending=False).iloc[0]
    count2 = df.groupby(colB).nunique()[colA].sort_values(ascending=False).iloc[0]

    if count1 == 0 or count2 == 0:
        relationship = "None"
    elif count1 > 1 and count2 > 1:
        relationship = "N:N"
    elif count1 == 1 and count2 == 1:
        relationship = "1:1"
    elif count1 > 1:
        relationship = "1:N"
    else:
        relationship = "N:1"

    return f"'{colA}','{colB}'\t{relationship}"

#### Bikeways

In [ ]:
bike_ciclabili = import_geojson("https://dati.comune.milano.it/dataset/ceda0264-24f3-4869-9a2d-411906f0abab/resource/56515ac3-e260-4ebb-bfce-698347f07e1e/download/bike_ciclabili.geojson")
bike_ciclabili

In [ ]:
bike_ciclabili["lunghezza"] = bike_ciclabili["lunghezza"].astype(int)
bike_ciclabili.dtypes

In [ ]:
print(keys_relationship(bike_ciclabili, "id_via", "id_amat"))
print(keys_relationship(bike_ciclabili, "id_via", "anagrafica"))
print(keys_relationship(bike_ciclabili, "anagrafica", "id_amat"))

In [ ]:
bike_lanes = bike_ciclabili["geometry"].get_coordinates().join(bike_ciclabili[["id_amat", "id_via", "anagrafica", "rete", "tipologia", "sede", "marcia", "lunghezza"]]) # explode to as many rows as points in the geometry
bike_lanes.columns = ["bike_lanes_" + col for col in bike_lanes.columns]

n_points_bike_lanes = bike_lanes.shape[0]
n_points_bike_lanes_cicle = bike_lanes[bike_lanes["bike_lanes_rete"] == "ciclabile"].shape[0]
n_points_bike_lanes_cicle_own_path = bike_lanes[bike_lanes["bike_lanes_tipologia"] == "ciclabile sede propria"].shape[0]

bike_lanes_x_min = bike_lanes['bike_lanes_x'].min()
bike_lanes_x_max = bike_lanes['bike_lanes_x'].max()
bike_lanes_y_min = bike_lanes['bike_lanes_y'].min()
bike_lanes_y_max = bike_lanes['bike_lanes_y'].max()
bike_lanes_extreme_sw = Point(bike_lanes_y_min, bike_lanes_x_min) # coordinates of the extremes of the cicle bike_lanes' map
bike_lanes_extreme_se = Point(bike_lanes_y_min, bike_lanes_x_max)
bike_lanes_extreme_nw = Point(bike_lanes_y_max, bike_lanes_x_min)
bike_lanes_size_x = geodesic(bike_lanes_extreme_sw, bike_lanes_extreme_se).km # size of the cicle bike_lanes' map in kilometers
bike_lanes_size_y = geodesic(bike_lanes_extreme_sw, bike_lanes_extreme_nw).km

print(f"Points all lanes:\t\t{n_points_bike_lanes}")
print(f"Points bikeways only:\t\t{n_points_bike_lanes_cicle}\t({perc(n_points_bike_lanes_cicle, n_points_bike_lanes)}% of total)")
print(f"Points protected bikeways only:\t{n_points_bike_lanes_cicle_own_path}\t({perc(n_points_bike_lanes_cicle_own_path, n_points_bike_lanes)}% of total)")
print(f"\nMinimal map surface needed:\t{round(bike_lanes_size_x, 2)}km x {round(bike_lanes_size_y, 2)}km")

In [ ]:
with open("data/raw/milan_open_data/bike_ciclabili.pickle", "wb") as file:
    pickle.dump(bike_ciclabili, file)

#### Bike Sharing Stations (not used yet)

In [ ]:
bikemi_stazioni = import_geojson("https://dati.comune.milano.it/dataset/cc065002-cd21-4dcb-b84f-bba2fd9e0c86/resource/2d0bafbc-1739-4a14-9e5a-71a648e1fc5b/download/bikemi_stazioni.geojson")
bikemi_stazioni

In [ ]:
bikemi_stazioni["stalli"] = bikemi_stazioni["stalli"].astype(int)
bikemi_stazioni["zd_attuale"] = bikemi_stazioni["zd_attuale"].astype(int)
# bikemi_stazioni["anno"] = bikemi_stazioni["anno"].astype(int)
bikemi_stazioni.dtypes

In [ ]:
# with open("data/raw/milan_open_data/bikemi_stazioni.pickle", "wb") as file:
#     pickle.dump(bikemi_stazioni, file)

### OpenStreetMap Network

In [ ]:
G = ox.graph_from_address("Milan, Italy",
    dist=10000,
    network_type="walk",
    simplify=True) # remove nodes which are not intersection or dead ends

city = ox.graph_to_gdfs(G, edges=False)

print(f"Nodes: {city.shape[0]}")

In [ ]:
with open("G_milan.pickle", "wb") as file:
    pickle.dump(G, file)

## Compute Driving Distances

In [ ]:
# For each point of the cicle lanes, find the nearest node in the street network - its "access point" to whole network

bike_lanes["nearest_node"] = bike_lanes.progress_apply(lambda row: ox.distance.nearest_nodes(G, Y=row["lanes_y"], X=row["lanes_x"]), axis=1)

with open("data/processed/milan_open_data/bike_lanes.pickle", "wb") as file:
    pickle.dump(bike_lanes, file)

In [ ]:
def compute_node_distances(G, points_of_interest, weight:str="length"):
    '''
    Given the entire graph and a list nodes identifiers corresponding to some points of interest
    Returns the length of the shortest path from each node to the closest point of interest
    '''

    node_distances = DictSmallest()

    for point in tqdm(range(points_of_interest.shape[0])):
        temp = nx.shortest_path_length(G, points_of_interest.iloc[point], weight=weight)
        node_distances.update(temp)
    
    coordinates_distances = {
        key: {"x": G.nodes[key]["x"], "y": G.nodes[key]["y"], "distance": node_distances[key]}
        for key in list(G.nodes())
        }
    
    return coordinates_distances

In [ ]:
# For each node in the network, compute the shortest driving distance to one of those that were identified as "access points" to the cicle lanes
# Unfortunately, we do not have space to store a matrix with all distances: we can use DictSmallest to solve the problem, but we'll have to repeat the computation at every variation of the list of points of interest

distances_bike_lanes = compute_node_distances(G, bike_lanes["nearest_node"])
with open('data/processed/distances/distances_bike_lanes.pickle', 'wb') as file:
    pickle.dump(distances_bike_lanes, file)

distances_bike_lanes_protected = compute_node_distances(G, bike_lanes.loc[bike_lanes["lanes_tipologia"] == "ciclabile sede propria", "nearest_node"])
with open('data/processed/distances/distances_bike_lanes_protected.pickle', 'wb') as file:
    pickle.dump(distances_bike_lanes_protected, file)

## Plot Isochrone

In [ ]:
# with open('data/raw/milan_open_data/bike_ciclabili.pickle', 'rb') as file:
#     bike_ciclabili = pickle.load(file)

# with open('data/processed/distances/distances_bike_lanes.pickle', 'rb') as file:
#     distances_bike_lanes = pickle.load(file)

# with open('data/processed/distances/distances_bike_lanes_protected.pickle', 'rb') as file:
#     distances_bike_lanes_protected = pickle.load(file)

In [ ]:
def isochrone(distances, geometry_data=None, geometry_on_hover:str=None, scale_min:int=0, scale_max:int=31, scale_step:int=1, scale_cmap:str="plasma", map_tiles:str="cartodb positron", dot_radius:int=50, dot_opacity:float=.3, start_zoom:int=12, start_location:list=[45.464098, 9.191926]):

    scale = range(scale_min, scale_max, scale_step)

    iso_colors = ox.plot.get_colors(n=len(scale), cmap=scale_cmap, start=0.3, return_hex=True)
    iso_colors.reverse()
    colormap = branca.colormap.LinearColormap(colors=iso_colors)
    colormap = colormap.to_step(index=scale)

    def color_mapping_function(val, iso_colors):
        for time, color in zip(scale, iso_colors):
            if val < time :
                return color
        return iso_colors[-1]
    
    colored_distances = distances.copy()
    for node in colored_distances:
        colored_distances[node]["color"] = color_mapping_function(colored_distances[node]["distance"], iso_colors)
    
    colormap = branca.colormap.LinearColormap(colors=iso_colors)
    colormap = colormap.to_step(index=scale)

    m = folium.Map(
        location=start_location,
        zoom_start=start_zoom,
        prefer_canvas=True,
        tiles=map_tiles
    )

    for val in colored_distances.values():
        folium.Circle(
            location=[val["y"], val["x"]],
            radius=dot_radius,
            stroke=False,
            fill=True,
            color=val["color"],
            fill_opacity=dot_opacity,
            interactive=False,
        ).add_to(m)
    colormap.add_to(m)

    if geometry_data is not None:
        for _, r in geometry_data.iterrows():
            sim_geo = gpd.GeoSeries(r["geometry"]).simplify(tolerance=0.001)
            geo_j = sim_geo.to_json()
            geo_j = folium.GeoJson(data=geo_j)
            if geometry_on_hover is not None:
                folium.Popup(r[geometry_on_hover]).add_to(geo_j)
            geo_j.add_to(m)

    return m

In [ ]:
isochrone(distances_bike_lanes, geometry_data=bike_ciclabili, geometry_on_hover="anagrafica", dot_opacity=.1, dot_radius=45, scale_min=0, scale_max=1500)

In [ ]:
isochrone(distances_bike_lanes_protected, geometry_data=bike_ciclabili.loc[bike_ciclabili["tipologia"] == "ciclabile sede propria"], geometry_on_hover="anagrafica", dot_opacity=.1, dot_radius=45, scale_min=0, scale_max=1500)